In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["NUM_THREADS"] = "4"
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["PYTHONNOUSERSITE"] = "1"
import numpy
from write_to_tf import * 
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta



2023-11-10 14:56:58.835807: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 14:56:58.861893: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 14:56:58.861909: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 14:56:58.861928: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 14:56:58.868201: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 14:56:58.868444: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

total cores: 255


In [2]:
import socket
hostname = socket.gethostname()
if "nird" in hostname:
    tf.config.threading.set_inter_op_parallelism_threads(1)

    data_loc = "/nird/projects/NS9600K/fslippe/data/"
    folder = "/nird/projects/NS9600K/data/modis/cao/MOD02/2020/ /nird/projects/NS9600K/data/modis/cao/MOD02/2021/ /nird/projects/NS9600K/data/modis/cao/MOD02/2023/"
if "mimi" in hostname:
    data_loc = "/uio/hume/student-u37/fslippe/data/"
    folder = "/scratch/fslippe/modis/MOD02/daytime_1km/ /scratch/fslippe/modis/MOD02/boundary_1km/ /scratch/fslippe/modis/MOD02/night_1km/ /scratch/fslippe/modis/MOD02/may-nov_2021/"



In [3]:
#folder = "/scratch/fslippe/modis/MOD02/daytime_1km/ /scratch/fslippe/modis/MOD02/boundary_1km/ /scratch/fslippe/modis/MOD02/night_1km/ /scratch/fslippe/modis/MOD02/may-nov_2021/"
patch_size = 256
bands = [29]
start = "20201001"
end = "20210430"
# start = "20210401"
# end = "20210430"
start = "20201201"
end = "20210430"
dates = generate_date_list(start, end)
start = "20211001"
end = "20211130"
dates.extend(generate_date_list(start, end))

start = "20201201"
end = "20201202"
dates = generate_date_list(start, end)
# start_converted = convert_to_day_of_year(start)
# end_converted = convert_to_day_of_year(end)
# print(start_converted)
# print(end_converted)
x, dates, masks = extract_1km_data(folder, bands=bands, date_list=dates, data_loc=data_loc)
x, dates, masks = zip(*[(xi, date, mask) for xi, date, mask in zip(x, dates, masks) if (xi.shape[0] > patch_size) and (xi.shape[1] > patch_size)])
x = list(x)
dates = list(dates)


['/nird/projects/NS9600K/data/modis/cao/MOD02/2020/', '/nird/projects/NS9600K/data/modis/cao/MOD02/2021/', '/nird/projects/NS9600K/data/modis/cao/MOD02/2023/']
['2020001', '2020002', '2020003', '2020004', '2020005', '2020006', '2020007', '2020008', '2020009', '2020010', '2020011', '2020012', '2020013', '2020014', '2020015', '2020016', '2020017', '2020018', '2020019', '2020020', '2020021', '2020022', '2020023', '2020024', '2020025', '2020026', '2020027', '2020028', '2020029', '2020030', '2020031', '2020032', '2020033', '2020034', '2020035', '2020036', '2020037', '2020038', '2020039', '2020040', '2020041', '2020042', '2020043', '2020044', '2020045', '2020046', '2020047', '2020048', '2020049', '2020050', '2020051', '2020052', '2020053', '2020054', '2020055', '2020056', '2020057', '2020058', '2020059', '2020060', '2020061', '2020062', '2020063', '2020064', '2020065', '2020066', '2020067', '2020068', '2020069', '2020070', '2020071', '2020072', '2020073', '2020074', '2020075', '2020076', '20

100%|██████████| 2/2 [00:59<00:00, 29.57s/it]


In [4]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["NUM_THREADS"] = "4"
os.environ["OMP_NUM_THREADS"] = "4"
import numpy as np 
import tensorflow as tf 

x = np.array([1,1,1])
patches = tf.image.extract_patches(images=x[0],
                                            sizes=[1,128,128, 1],
                                            strides=[1,128,128, 1],
                                            rates=[1,1,1, 1],
                                            padding="VALID")

InvalidArgumentError: {{function_node __wrapped__ExtractImagePatches_device_/job:localhost/replica:0/task:0/device:CPU:0}} input must be 4-dimensional[] [Op:ExtractImagePatches]

In [6]:
bands = [1]
patch_size = 128
import autoencoder
import importlib
importlib.reload(autoencoder)
from autoencoder import * 
autoencoder = SimpleAutoencoder(len(bands), patch_size, patch_size)
all_patches = []
i=0
tot = len(x)
for (image, mask) in zip(x, masks):
    print("%s/%s" %(i, tot))
    patches, idx, n_patches = autoencoder.extract_patches(image, mask, mask_threshold=0.95)  # Assuming this function extracts and reshapes patches for a single image
    all_patches.append(patches)
    i+=1

patches = np.concatenate(all_patches, axis=0)

0/3
no EXTRACT


InvalidArgumentError: {{function_node __wrapped__ExtractImagePatches_device_/job:localhost/replica:0/task:0/device:CPU:0}} input must be 4-dimensional[] [Op:ExtractImagePatches]

In [4]:
len(patches)

147641

In [5]:
# TRAIN TEST SPLIT
patches, val_data = train_test_split(patches, test_size=0.15, random_state=42, shuffle=True)

In [6]:
###  NORMALIZE PATCHES 
model_run_name = "dnb_l95_z50_ps128_(29)_%s-%s" %("cao_months_202012", "202111")
print(model_run_name)
max_val = np.max(patches)
patches = patches / max_val
val_data = val_data / max_val

dnb_l95_z50_ps128_(29)_cao_months_202012-202111


In [9]:
patches_per_file = 50000 

for i in range(0, len(patches), patches_per_file):
    print(i)
    chunk = patches[i: i+patches_per_file]
    write_tfrecord(f'/scratch/fslippe/modis/MOD02/training_data/tf_data/normalized_trainingpatches_{model_run_name}_{i//patches_per_file}.tfrecord', chunk)

np.save("/uio/hume/student-u37/fslippe/data/models/max_vals_%s.npy" %(model_run_name), max_val)
np.save("/scratch/fslippe/modis/MOD02/training_data/tf_data/normalized_valpatches_%s" %(model_run_name), val_data)

0
50000
100000
